In [3]:
from shiny import App, render, ui
import os, sys
import numpy as np
from scipy.interpolate import UnivariateSpline

# return a list of lines from a url
# if running in pyodide, use pyodide.open_url
def url_open(url):
    if "pyodide" in sys.modules:
        print("Running in pyodide, using pyodide.open_url") 
        import pyodide # type: ignore
        lines = pyodide.http.open_url(url).readlines()
    else:
        from urllib.request import urlopen
        if os.path.exists("weight.csv"):
            print("Using local copy of weight.csv")
            file = open('weight.csv', 'r')
            lines = file.readlines()
        else:
            print("Downloading weight.csv from Google Sheets")
            lines = [line.decode("utf-8").strip() + "\n" for line in urlopen(url)]
            file = open('weight.csv', 'w')
            for line in lines:
                file.write(line)
            file.close()

    print(f"Read {len(lines)} lines")
    return lines


In [131]:

class WeightData:
    def __init__(self, dates = None, weights = None, spline = None, url = None):
        if url is None:
            self.url = "https://docs.google.com/spreadsheets/d/151vhoZ-kZCnVfIQ7h9-Csq1rTMoIgsOsyj_vDRtDMn0/export?gid=1991942286&format=csv"

        if dates is None or weights is None:
            self.dates, self.weights = self._getData()
            order = np.argsort(self.dates)
            self.dates = self.dates[order]
            self.weights = self.weights[order]
        else:
            self.dates = dates
            self.weights = weights
        
        self.spline = UnivariateSpline(self.dates.astype("float"), self.weights, s=0)

    def __repr__(self):
        return f"{self.dates.size} data points from {self.dates[0]} to {self.dates[-1]}"
    
    def __len__(self):
        return len(self.dates)

    def __getitem__(self, index):
        return WeightData(self.dates[index], self.weights[index], self.spline, self.url)

    def _getData(self):
        lines = url_open(self.url)

        dates = []
        weights = []
        for idx, line in enumerate(lines):
            date, weight = line.split(",")[0:2]
            if date == "date":
                continue
            elif len(dates) > 0:
                if date == dates[-1]:
                    if weight == weights[-1]:
                        s = "are the same"
                    else:
                        s = "differ"

                    print(f"Duplicate date {date} at line {idx} and {idx-1} and weights {s}, skipping line {idx}")
                    continue

            dates.append(date)
            weights.append(weight)

        return (
            np.array(dates, dtype="datetime64"),
            np.array(weights, dtype="float"),
            )

    def filter(self, start, end = None):
        start = np.datetime64(start)
        if end is None:
            end = np.datetime64("today")
        
        inRng = (self.dates >= start) & (self.dates <= end)
        return WeightData(self.dates[inRng], self.weights[inRng], self.spline, self.url)
    
    def days(self, days):
        return self.filter(self.dates[-1] - np.timedelta64(days, 'D'))

    def last(self, n):
        return WeightData(self.dates[-n:], self.weights[-n:], self.spline, self.url)

In [137]:
data = WeightData()
data.spline(np.datetime64("2023-01-01 00:00:00"))

Using local copy of weight.csv
Read 981 lines
Duplicate date 2022-05-08 09:55:00 at line 385 and 384 and weights differ, skipping line 385
Duplicate date 2022-12-15 13:18:00 at line 826 and 825 and weights differ, skipping line 826
Duplicate date 2022-12-15 13:18:00 at line 827 and 826 and weights differ, skipping line 827
Duplicate date 2022-12-16 22:07:00 at line 831 and 830 and weights are the same, skipping line 831


TypeError: Cannot cast array data from dtype('<M8[s]') to dtype('float64') according to the rule 'safe'

In [41]:
data.dates.astype(np.float64)

array([1.45653120e+09, 1.47182400e+09, 1.49238720e+09, 1.54440000e+09,
       1.55165760e+09, 1.55243520e+09, 1.55247840e+09, 1.55304000e+09,
       1.55308320e+09, 1.55381760e+09, 1.55386080e+09, 1.55485440e+09,
       1.55489760e+09, 1.55787840e+09, 1.55792160e+09, 1.55848320e+09,
       1.56055680e+09, 1.56245760e+09, 1.56254400e+09, 1.56608640e+09,
       1.56695040e+09, 1.58401440e+09, 1.58414400e+09, 1.58487840e+09,
       1.58760000e+09, 1.59045120e+09, 1.59049440e+09, 1.59105600e+09,
       1.59109920e+09, 1.59114240e+09, 1.59118560e+09, 1.59122880e+09,
       1.59127200e+09, 1.59131520e+09, 1.59135840e+09, 1.59140160e+09,
       1.59144480e+09, 1.59157440e+09, 1.59161760e+09, 1.59174720e+09,
       1.59179040e+09, 1.59209280e+09, 1.59213600e+09, 1.59239520e+09,
       1.59308640e+09, 1.59369120e+09, 1.59395040e+09, 1.59498720e+09,
       1.59649920e+09, 1.59654240e+09, 1.59658560e+09, 1.59671520e+09,
       1.59675840e+09, 1.59710400e+09, 1.59732000e+09, 1.59740640e+09,
      